In [ ]:
# Millions to miss out on the net
# aust addresses un security council over iraq

In [1]:
import requests
import os
from tqdm import tqdm
import spacy
nlp = spacy.load("en")

In [2]:
host = "http://0.0.0.0:8890/correct"

def call_gec(data):
    resp = requests.post(host, json=data)
    res = resp.json()
    return res

In [3]:
data_path = '/home/citao/github/BBC-Dataset-News-Classification/dataset/data_files'
domains = ['business', 'entertainment', 'politics', 'sport', 'tech']

In [ ]:
corpus = {d: [] for d in domains}

for domain in domains:
    domain_path = os.path.join(data_path, domain)
    for f in os.listdir(domain_path):
        full_path = os.path.join(domain_path, f)
        try:
            doc = nlp(open(full_path).read())
            corpus[domain].extend([i.text for i in doc.sents])
        except:
            print(full_path)
    print(domain, len(corpus[domain]))
            

In [ ]:
import json

json.dump(corpus, open('./bbc_news.json', 'w'))

In [199]:
# text = "England coach Andy Robinson is already without centre Will Greenwood and flanker Richard Hill while fly-half Jonny Wilkinson is certain to miss at least the first two games."
# text = "Hi, Guibin! My namme is Citao. The marked was closed yestreday. (This email are sent from OnMail.)"
# text = "Millions to miss out on the net"
# text = "The marked is closed today."

text = """
Hi, 

My namme is Citao Wu  (EID: cw39729) and I do admitted into the Computer Online Master prograssm 2021 spring. In my status panel, I saw a requirement about "Required immunizations".

My question is whether I still need to meets this requirement or not, even thoxugh my program is fully online. 

If no, could you please help me to remove this requirements from my application?

Thanks,

"""
# text = "I want go school"
# text = "I I want go to school"
data = {
    'text': text,
    'iterations': 3,
    'min_probability': 0.5,
    'min_error_probability': 0.7,
    'case_sensitive': True,
    'languagetool_post_process': True,
    'languagetool_call_thres': 0.7,
    'whitelist': ['Citao', 'Guibin', 'Onmail'],
    'with_debug_info': True,
}

result = call_gec(data)

print(result['debug_info'])
print(result['input'])
print(result['output'])
print(result['corrections'])


================================== Sentence_2 ==================================
[GECToR Model]
<Iteration 1>
Sentence Error Probability: 0.9584169983863831
# From #  My namme is Citao Wu
#  To  #  My name is Citao Wu .
--------------------------------------------------------------------------------
__START__                     $KEEP                         0.9989811778068542
My                            $KEEP                         0.9839291572570801
namme                         $REPLACE_name                 0.9964216947555542
is                            $KEEP                         0.9967592358589172
Citao                         $KEEP                         0.9921404719352722
Wu                            $APPEND_.                     0.5396763682365417
--------------------------------------------------------------------------------

<Iteration 2>
Sentence Error Probability: 0.002291475422680378
# From #  My name is Citao Wu .
#  To  #  My name is Citao Wu .
----------------

In [ ]:
for domain in domains:
    print(domain)
    right_data = record[domain]['pass']
    wrong_data = record[domain]['wrong']
    print(len(right_data), len(wrong_data))

In [ ]:
for sent in record['tech']['wrong']:
    print([sent])
    break

In [ ]:
import json

total = json.load(open('./bbc_news_result_2_0.8_0.9_False.json'))

i = 0

for min_probability in [0.5, 0.6, 0.7, 0.8]:
    for min_error_probability in [0.7, 0.8, 0.9]:
        for add_spell_check in [True, False]:
            fn = './result/bbc_news_result_2_{}_{}_{}.json'.format(min_probability, min_error_probability, str(add_spell_check))
            tmp = {domain: total[domain][5000*i:5000*(i+1)] for domain in domains}
            print(fn)
            json.dump(tmp, open(fn, 'w'))
            i+=1


In [ ]:
import pandas as pd

source_rows = []

for min_probability in [0.5, 0.6, 0.7, 0.8]:
    for min_error_probability in [0.7, 0.8, 0.9]:
        for add_spell_check in [True, False]:
            fn = './result/bbc_news_result_2_{}_{}_{}.json'.format(min_probability, min_error_probability, str(add_spell_check))
            tmp = json.load(open(fn))
            result = {
                'cased': {domain: {'right':[], 'wrong':[]} for domain in domains},
                'uncased': {domain: {'right':[], 'wrong':[]} for domain in domains}
            }
            print(fn)
            for domain in domains:
                for ori_sent, cor_sent, correction in tmp[domain]:
                    if correction == []:
                        result['cased'][domain]['right'].append(ori_sent)
                        result['uncased'][domain]['right'].append(ori_sent)
                    else:
                        result['cased'][domain]['wrong'].append(ori_sent)
                        uncased_correction = [c for c in correction if c[0].lower()!=c[1].lower()]
                        if uncased_correction:
                            result['uncased'][domain]['wrong'].append(ori_sent)
                        else:
                            result['uncased'][domain]['right'].append(ori_sent)
            
            for case_type in ['cased', 'uncased']:
                row = {
                    'min_probability': min_probability,
                    'min_error_probability': min_error_probability,
                    'add_spell_check': add_spell_check,
                    'cased_type': case_type,
                }
                row.update({d: len(result[case_type][d]['right'])/5000.0 for d in domains})
                source_rows.append(row)
            
                
            
            

In [ ]:
df = pd.DataFrame(source_rows)

In [ ]:
df

In [ ]:
df.to_csv('~/eval_pos.csv', index=False)

In [ ]:
for domain in domains:
    df[df['min_probability']==0.]

In [ ]:
import numpy as np
1-np.average(df[df['cased_type']=='uncased'].iloc[:, 4:].to_numpy())

In [30]:
path='/home/citao/github/gector/dataset/'
fn = 'wil.ABCN.dev.gold.bea19.0'
source_list = [i.strip() for i in open(path+fn+'.source').readlines()]
target_list = [i.strip() for i in open(path+fn+'.target').readlines()]

diff = 0
for s, t in zip(source_list, target_list):
    if s!=t:
        diff+=1

print(diff, len(source_list), len(target_list))

2819 4384 4384


In [38]:
4384-2819

1565

In [35]:
for s, t in zip(source_list[:10], target_list):
    print(s)
    print(t)
    print()

It 's difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .
It 's difficult to answer the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .

When I was younger I used to say that I wanted to be a teacher , a saleswoman and even a butcher .. I do n't know why .
When I was younger , I used to say that I wanted to be a teacher , a saleswoman and even a butcher . I do n't know why .

I would like to study Psychology because one day I would open my own psychology office and help people .
I would like to study Psychology , because one day I would like to open my own psychology clinic and help people .

It 's difficult because I 'll have to study hard and a lot , but I think that if you like a subject , you 'll study it easier .
It 's difficult because I 'll have to study hard and a lot , but I think that if you like a subject , you 'll study it more easily .



In [40]:
25000+1565

26565

In [41]:
2819 / (2819+26565)

0.095936564116526

In [49]:
import Levenshtein
from difflib import SequenceMatcher
from nltk.corpus import words


NLTK_COMMON_WORDS = {w:1 for w in words.words()}

# 1: spelling error
# 2: grammar error
def spell_or_grammer_error(correction):
    ori, cor, _ = correction
    if len(ori.split()) != len(cor.split()):
        return 2
    # dist = Levenshtein.distance(ori, cor)
    # print(dist)
    seq = SequenceMatcher(None, ori, cor)
    ratio = seq.ratio()
    print(ratio)

    # if dist <= 3:
    if ratio > 0.8 and ori not in NLTK_COMMON_WORDS:
        return 1
    else:
        return 2

correction = ['are', 'is', [10, 15]]
spell_or_grammer_error(correction)

0.0


2